<a href="https://colab.research.google.com/github/PhiCtl/Flower_detection/blob/main/Object_detection_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Acknowledgments 
* https://github.com/ultralytics/yolov5

# What this notebook allows
1. To train the following neural networks :
    * Any YOLO network from ultralytics repository
2. To visualize their predictions
3. To evaluate their performance
4. To load a model and make inference

# 1 Training

## A) Upload necessary git repositories and packages
 * My repo
 * Ultralytics'
 * Object detection metrics

In [ ]:
%%shell
# My own repo
#rm -r Flower_detection
git clone https://github.com/PhiCtl/Flower_detection.git
cd Flower_detection/src
cp myClasses.py ../../
cp myTransforms.py ../../
cp myUtils.py ../../
cp myTrainingUtils.py ../..

In [ ]:
%%shell
git clone https://github.com/ultralytics/yolov5  # master branch (default)
cd yolov5
pip install -r requirements.txt

In [ ]:
%%shell
# ref https://github.com/rafaelpadilla/Object-Detection-Metrics#how-to-use-this-project
git clone https://github.com/rafaelpadilla/Object-Detection-Metrics.git

## B) Imports

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
import torchvision, cv2, time, copy
import torchvision.transforms as T
import torch.functional as F
from myUtils import*
from myTrainingUtils import eval_custom_YOLO
from myClasses import FlowerDetectionDataset, Rescale
from myTransforms import *
from myTrainingUtils import*

# ref https://github.com/pytorch/vision/blob/master/references/detection/
# ref https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

################################################################################
# GETTING STARTED
################################################################################

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


## C) Load data

### Defining Transforms

In [ ]:
myTransform = Rescale((640,640)) # rescale for training
transforms_train = T.Compose([T.ToTensor(), T.ColorJitter(0.1, 0.1), T.Normalize(mean=MEAN_Imagenet, std= STD_Imagenet)])
transforms_test = get_img_transformed(train=False)

### To rescale the entire dataset and write in YOLO format

In [ ]:
def write_lower_res(dataset, train = True, dir = '/content/drive/MyDrive/GBH/rescaled/'):
  if train:
        path = dir + 'data_train/'
  else:
        path = dir + 'data_test/'

  for (img, target), i in zip(dataset, range(len(dataset))):
        img_name, name = path + 'images/' + dataset.imgs[i], dataset.imgs[i][:-4]
        ny, nx = img.shape[1:]
        file_name = path + 'labels/' + name + '.txt'

        cv2.imwrite(img_name, img.numpy().transpose(1,2,0)*255)

        f = open(file_name,'w+') # open file in w mode
        for label, bbox in zip(target['labels'], target['boxes']):
            # compute scaled center box
            cx = (bbox[2]+bbox[0])/(2*nx)
            cy = (bbox[3]+bbox[1]) / (2*ny)
            f.write("{} {} {} {} {}\r\n".format(label-1, cx, cy, (bbox[2] -bbox[0])/nx, (bbox[3]-bbox[1])/ny))
        f.close()



In [ ]:
write_lower_res(dataset)
write_lower_res(dataset_test, train=False)

### Load train and test datasets

In [133]:
################################################################################
# LOAD DATASET         
################################################################################

dataset = FlowerDetectionDataset('/content/drive/MyDrive/GBH/data_train/images/', json_file_root='/content/drive/MyDrive/GBH/labels/export1m.json', custom_transforms=None, transforms=transforms_train)
data_loader = DataLoader(
    dataset, batch_size=5, shuffle=True, num_workers=2,
    collate_fn=collate_fn)

dataset_test = FlowerDetectionDataset('/content/drive/MyDrive/GBH/data_test/images/', json_file_root='/content/drive/MyDrive/GBH/labels/export2m.json', custom_transforms=None, transforms=transforms_test)
data_loader_test = DataLoader(
    dataset_test, batch_size=2, shuffle=False, num_workers=2,
    collate_fn=collate_fn)

### To write into YOLO compatible format
Note: Only done once

In [10]:
import cv2

def write_YOLOformat(dataset, dir='/content/drive/MyDrive/GBH/final_test/', root='/content'):
    """
    Write data bouding boxes in format <label><center_x> <center_y> <widht> <height>
    and store images in correct dimensions for train
    :param dataset:
    :param train: (bool) if training set or test set
    :param dir: directory containing the folders /data_train an /data_test
    :param root: root directory
    :return:
    """
    path = dir
    # if train:
    #     path = dir + 'data_train/'
    # else:
    #     path = dir + 'data_test/'

    # <class_name> <x_center> <y_center> <width> <height>
    for (img, target), i in zip(dataset, range(len(dataset))):
        name = dataset.imgs[i][:-4]
        ny, nx = img.shape[1:]
        file_name = path + 'labels/' + name + '.txt'
        #img_name = path + 'images_YOLO/' + name + '.jpg'

        #cv2.imwrite(img_name, img.numpy().transpose(1,2,0)*255)

        f = open(file_name,'w+') # open file in w mode
        for label, bbox in zip(target['labels'], target['boxes']):
            # compute scaled center box
            cx = (bbox[2]+bbox[0])/(2*nx)
            cy = (bbox[3]+bbox[1]) / (2*ny)
            f.write("{} {} {} {} {}\r\n".format(label-1, cx, cy, (bbox[2] -bbox[0])/nx, (bbox[3]-bbox[1])/ny))
        f.close()



In [ ]:
write_YOLOformat(dataset, train=True)
write_YOLOformat(dataset_test, train=False)

To undo, launch batch snippet below.
Of course, paths must be changed.

In [ ]:
%%bash
cd /content/drive/MyDrive/GBH/data_test/
rm -r labels
cd /content/drive/MyDrive/GBH/data_train/
rm -r labels

In [ ]:
%%bash
cd /content/drive/MyDrive/GBH/results/
rm -r detections groundtruths
mkdir detections
mkdir groundtruths

In [ ]:
write_YOLOformat(dataset, train=True)

In [ ]:
write_YOLOformat(dataset_test, train=False)

## D) Compute data augmented training set for YOLO training

In [ ]:
root_img = '/content/drive/MyDrive/GBH/data_train/images'
root_img_tr = '/content/drive/MyDrive/GBH/data_train/images_YOLO'
img_list = list(sorted(os.listdir(root_img)))
if '.ipynb_checkpoints' in img_list: img_list.remove('.ipynb_checkpoints')

transform = T.Compose([T.ToTensor(), 
                              T.ColorJitter(0.1, 0.1), 
                              T.Normalize(mean=MEAN_Imagenet, std= STD_Imagenet)])

for im_name in img_list:
  img = cv2.imread(os.path.join(root_img, im_name))
  img_tr = transform(img)
  img_tr = img_tr.numpy().transpose(1,2,0) * 255
  cv2.imwrite(os.path.join(root_img_tr, im_name), img_tr)


## E) Train model

### Defining our model
We will try YOLO object detector which is mainly used for mobile and real time applications. It comes in different flavours, namely YOLOv3 and YOLOv5 derivatives.

### Built in training
Refer to https://github.com/ultralytics/yolov5 for more details.

In [ ]:
%%shell
cd yolov5
python train.py  --batch 6 --epochs 20 --data /content/drive/MyDrive/GBH/rescaled/train.yaml --weights yolov5x.pt

# 2. Visualize results

## A) Pick one example on test set
Again, look at  https://github.com/ultralytics/yolov5 for more details.

In [ ]:
%%shell
cd yolov5
python detect.py --source /content/drive/MyDrive/GBH/data_test/images/20210419_144007.jpg --weights /content/drive/MyDrive/GBH/models/yolo_06062021_1457/best.pt --conf 0.15

## B) Save model
 

In [ ]:
import os
os.chdir('/content/yolov5/')
from utils.plots import plot_results 
plot_results(save_dir='runs/train/exp4')  # plot results.txt as results.png

In [ ]:
%%shell
mkdir /content/drive/MyDrive/GBH/models/yolo_06062021_1457
# mkdir /content/drive/MyDrive/GBH/results/yolov5x
mkdir /content/drive/MyDrive/GBH/results/yolov5x/06062021_1457
cp /content/yolov5/runs/train/exp4/weights/* /content/drive/MyDrive/GBH/models/yolo_06062021_1457
cd /content/yolov5/runs/train/exp4
cp F1_curve.png PR_curve.png P_curve.png R_curve.png results.png labels.jpg /content/drive/MyDrive/GBH/results/yolov5x/06062021_1457

# 3. Evaluate model on validation set

## A) Download model

In [ ]:
import time, torch
import torchvision.transforms as T

device = torch.device('cpu')
weights_path = '/content/drive/MyDrive/GBH/models/yolo_18052021_1550/best.pt' # TODO : change path

# Load trained model
model_type = 'yolov5x'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)


## B) Make predictions

In [143]:
predictions = []
path = '/content/drive/MyDrive/GBH/final_test/images/' # path to the validation or test set
with torch.no_grad():
  for i in range(len(dataset)):
    
    img_path = path + dataset.imgs[i]
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image)

    target = {}
    target['boxes'] = results.xyxy[0][:,:4]
    target['scores'] = results.xyxy[0][:,4].flatten()
    target['labels'] = results.xyxy[0][:,5].flatten()

    predictions.append(target)


Check whether at least one flower has been predicted per frame.

In [ ]:
for i, pred in enumerate(predictions):
  if len(pred['boxes']) < 1:
    print(i)

## C) Write to files

In [153]:
def write(dataset, prediction=None, gt=True, initial_dir='/content/drive/MyDrive/GBH/results', root='/content'):
    """
    Writes prediction or groundtruth bboxes to files
    :param dataset: (torch.Dataset)
    :param prediction: (dic) output of model
    :param gt: (bool) if we want to write groundtruth to files
    :param initial_dir : should contain the two following subfolders: groundtruths and detections
    :param root: root directory
   """

    if gt:
        path = initial_dir + '/groundtruths'
        os.chdir(path)
        # <class_name> <left> <top> <right> <bottom>
        for (_, target), i in zip(dataset, range(len(dataset))):
            name = dataset.imgs[i]
            file_name = name + '.txt'
            f = open(file_name, 'w+')  # open file in w mode
            for label, bbox in zip(target['labels'], target['boxes']):
                f.write("{} {} {} {} {}\r\n".format(label, bbox[0], bbox[1], bbox[2], bbox[3]))
            f.close()

    if prediction is not None:
        path = initial_dir + '/detections'
        os.chdir(path)
        # <class_name> <confidence> <left> <top> <right> <bottom>
        for pred, (_, target), i in zip(prediction, dataset, range(len(dataset))):
            name = dataset.imgs[i]
            file_name = name + '.txt'
            f = open(file_name, 'w+')
            if len(pred['scores']) >=1 :
              for label, score, bbox in zip(pred['labels'], pred['scores'], pred['boxes']):
                  f.write("{} {} {} {} {} {}\r\n".format(int(label), score, int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])))
            else:
              f.write("1 1 0 0 0 0 \r\n") # arbitrarily false detection
            f.close()

    os.chdir(root)


In [154]:
write(dataset, predictions, gt=True)

## D) Evaluate
TODO : create a folder to store predictions

In [ ]:
%%shell
mkdir /content/drive/MyDrive/GBH/results/TEST_FRCNN_mobilenetv3_320/
cd Object-Detection-Metrics/
python pascalvoc.py  -gt /content/drive/MyDrive/GBH/results/groundtruths/ -det /content/drive/MyDrive/GBH/results/detections/ -gtformat xyrb -detformat xyrb -sp /content/drive/MyDrive/GBH/results/TEST_FRCNN_mobilenetv3_320/

# 4. Inference

## A) Load model

In [ ]:
import time, torch
import torchvision.transforms as T

device = torch.device('cpu')
weights_path = '/content/drive/MyDrive/GBH/models/yolo_18052021_1550/best.pt' # TODO : change path

# Load trained model
model_type = 'yolov5x'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)


## B) Load image

In [164]:
import cv2

# Load and transform image
img = cv2.imread('/content/drive/MyDrive/GBH/final_test/images/pic8_RGB.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (640,480))


## C) Compute inference time and make predictions

In [ ]:
model.conf = 0.25 # set model confidence score if needed
tot_time = 0
for i in range(3):
  t1 = time.time()
  results = model(img)
  tot_time += (time.time() - t1)
print("Mean elapsed time : {}s".format(tot_time/3))

In [ ]:
results.print()

## D) Visualize prediction

In [117]:
target = {}
target['boxes'] = results.xyxy[0][:,:4]
target['scores'] = results.xyxy[0][:,4].flatten()
target['labels'] = results.xyxy[0][:,5].flatten()


In [120]:
import math
def draw_bboxes(image, img_name, target, path ='/content/drive/MyDrive/GBH/final_res/', thresh = 0.3):
    img = image.copy()
    for [xm,ym,xM,yM], label, score in zip(target["boxes"], target["labels"], target["scores"]):
      c = ()
      font = cv2.FONT_HERSHEY_SIMPLEX
      if int(label) == 0: 
        c = (255,0,0)
        
      else: c = (0,255,0)
      if score > thresh :
        img = cv2.rectangle(img, (xm,ym), (xM,yM), c, 2)
        img = cv2.putText(img, str(math.trunc(score.item() * 100)) + '%', (xm,yM), font, 0.5, c, 1, cv2.LINE_AA )
    path += img_name
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, img)
    cv2_imshow(img) ## This line works only for Colab usage

In [ ]:
draw_bboxes(img_or, 'pic8.jpg', results[0], thresh = 0.5)